In [287]:
import importlib
from pybit.unified_trading import HTTP
import os
from utils import utils
importlib.reload(utils)
from datetime import datetime

class BybitWrapper():

    def __init__(self, demo=False, api_key=None, api_secret=None):
        print(f'Wrapper Activated. Demo Mode == {demo}')
        self.demo = demo
        
        if self.demo:   
            self.api_key = api_key or os.getenv("BYBIT_API_KEY_TEST")
            self.api_secret = api_secret or os.getenv("BYBIT_API_SECRET_TEST")
            self.session = HTTP(api_key=self.api_key, api_secret=self.api_secret, demo=demo, log_requests=True)
        else:
            self.api_key = api_key or os.getenv("BYBIT_API_KEY")
            self.api_secret = api_secret or os.getenv("BYBIT_API_SECRET")
            self.session = HTTP(api_key=self.api_key, api_secret=self.api_secret, demo=demo, log_requests=True)
        

    ###########################################################################
    ##########################  Account Data   ################################
    ########################################################################### 

    def transaction_log(self, account_type='UNIFIED', market=None, coin=None, limit=50):
        all_transactions = pd.DataFrame()  # Initialize an empty DataFrame
        cursor = None
        
        while True:
            # Make the API call with the current cursor
            response = self.session.get_transaction_log(
                accountType=account_type,
                category=market,
                baseCoin=coin,
                limit=limit,
                cursor=cursor
            )
            
            # Parse the current page of transactions and concatenate to the main DataFrame
            page_transactions = utils.parse_transaction_log(response)
            all_transactions = pd.concat([all_transactions, page_transactions], ignore_index=True)
            
            # Check for the next page cursor
            cursor = response.get('result', {}).get('nextPageCursor')
            if not cursor:
                break  # No more pages, exit loop

        # Return the combined transactions as a DataFrame
        return all_transactions


    def wallet_balance(self, account_type: str = 'UNIFIED', coin: str = None):
        
        response = self.session.get_wallet_balance(accountType=account_type, coin=coin)
        
        return utils.parse_wallet_balance(response)
    
    def get_coin_balance(self, account_type: str = 'UNIFIED', coin: str = None, member_id: str = None, with_bonus: int = 0):
        
        if self.demo:
            raise RuntimeError("This operation is not allowed in demo mode.")
        else:
            response = self.session.get_coins_balance(
                accountType=account_type,
                coin=coin,
                memberId=member_id,
                withBonus=with_bonus
            )
            return utils.parse_coin_balance(response=response)
    
    def get_api_details(self):
        if self.demo:
            raise RuntimeError("This operation is not allowed in demo mode.")
        else:
            response=self.session.get_api_key_information()
            return response
        
    ###########################################################################
    ##########################  Market Data   #################################
    ###########################################################################    
    

    # Market Data Endpoints (Common for Spot and Perpetual)
    def get_orderbook(self, ticker: str, category: str, limit: int = 100):
        response=self.session.get_orderbook(category=category, symbol=ticker, limit=limit)
        return utils.parse_orderbook(response=response)
    
    def get_candles(self, market, ticker, interval: str = "60", limit: int = 10):
        response=self.session.get_kline(category=market, symbol=ticker, interval=interval, limit=limit)
        return utils.parse_klines(response)


    ###########################################################################
    ##########################  Spot Order Management   #######################
    ###########################################################################


    def build_spot_market_order_payload(self,
                                ticker: str, 
                                side: str, 
                                qty: float, 
                                execution_type: str = 'IOC', 
                                annotations: str = None) -> dict:
        """
        Builds the payload for a market order on the spot market.

        :param ticker: The trading pair symbol (e.g., 'BTCUSDT').
        :param side: The side of the order ('Buy' or 'Sell').
        :param qty: The quantity of the asset to buy or sell.
        :param execution_type: The order execution type ('GTC', 'IOC', etc.). Defaults to 'GTC'.
        :param annotations: Optional annotations for the order.
        :return: A dictionary payload for the order.
        """

        # Validate side parameter
        if side not in ['Buy', 'Sell']:
            raise ValueError("Invalid side, must be 'Buy' or 'Sell'.")

        # Validate execution_type parameter
        valid_execution_types = ['GTC', 'IOC', 'FOK']
        if execution_type not in valid_execution_types:
            raise ValueError(f"Invalid execution_type, must be one of {valid_execution_types}.")

        # Automate annotations if not provided
        if annotations is None:
            current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
            annotations = f"{side}_{(str(qty).replace('.', '-'))}_{ticker}_{current_time}"

        payload = {
            "category": "spot",
            "symbol": ticker,
            "side": side,
            "orderType": "Market",
            "qty": qty,
            "timeInForce": execution_type,
            "orderLinkId": annotations,
            "isLeverage": 0,
            "orderFilter": "Order"
        }

        return payload

    # Spot Market Endpoints
    def place_spot_market_order(self, payload: dict):
        """
        Places a market order on the spot market using a pre-built payload.

        :param payload: A dictionary containing the order details.
        :return: The API response from placing the order.
        """
        try:
            self.session.place_order(**payload)
            
        except Exception as e:
            print(f"Error placing order: {e}")
            return None

        return self.spot_order_history(ticker=payload.get('symbol'),limit=1)
    
    def cancel_spot_order(self, symbol: str, order_id: str):
        pass

    def spot_order_history(self, market: str = 'spot', ticker = None, limit: int = 100):
        
        
        response = self.session.get_order_history(category=market, 
                                                  symbol=ticker,
                                                  limit=limit
                                                  )
        
        
        return utils.parse_order_history(response)
    

    ###########################################################################
    ####################  Derivatives Position Management  ####################
    ########################################################################### 

    def leverage(self, market='linear', ticker: str = None, buy_leverage: str = None, sell_leverage: str = None): 
        
        response = self.session.set_leverage(category=market,
                                             symbol=ticker,
                                             buyLeverage=buy_leverage,
                                             sellLeverage=sell_leverage)
        
        return response 


    def build_perp_market_order_payload(self,
                                ticker: str, 
                                side: str, 
                                qty: float, 
                                execution_type: str = 'IOC', 
                                annotations: str = None) -> dict:
        """
        Builds the payload for a market order on the spot market.

        :param ticker: The trading pair symbol (e.g., 'BTCUSDT').
        :param side: The side of the order ('Buy' or 'Sell').
        :param qty: The quantity of the asset to buy or sell.
        :param execution_type: The order execution type ('GTC', 'IOC', etc.). Defaults to 'GTC'.
        :param annotations: Optional annotations for the order.
        :return: A dictionary payload for the order.
        """

        # Validate side parameter
        if side not in ['Buy', 'Sell']:
            raise ValueError("Invalid side, must be 'Buy' or 'Sell'.")

        # Validate execution_type parameter
        valid_execution_types = ['GTC', 'IOC', 'FOK']
        if execution_type not in valid_execution_types:
            raise ValueError(f"Invalid execution_type, must be one of {valid_execution_types}.")

        # Automate annotations if not provided
        if annotations is None:
            current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
            annotations = f"{side}_{(str(qty).replace('.', '-'))}_{ticker}_{current_time}"

        payload = {
            "category": "linear",
            "symbol": ticker,
            "side": side,
            "orderType": "Market",
            "qty": qty,
            "timeInForce": execution_type,
            "orderLinkId": annotations,
            "isLeverage": 0,
            "orderFilter": "Order"
        }

        return payload

    # Spot Market Endpoints
    def place_perp_market_order(self, payload: dict):
        """
        Places a market order on the spot market using a pre-built payload.

        :param payload: A dictionary containing the order details.
        :return: The API response from placing the order.
        """
        try:
            self.session.place_order(**payload)
            
        except Exception as e:
            print(f"Error placing order: {e}")
            return None

        return self.perp_order_history(ticker=payload.get('symbol'),limit=1)
    
    def perp_order_history(self, market: str = 'linear', ticker = None, limit: int = 100):
        
        
        response = self.session.get_order_history(category=market, 
                                                  symbol=ticker,
                                                  limit=limit
                                                  )
        
        
        return utils.parse_order_history(response)
    
    def cancel_all_ordera(self, market=None):
        response = self.session.cancel_all_orders(category=market)

        return response

    def positions(self, market: str = 'linear', ticker: str = None, settleCoin: str = 'USDT', limit: int = 20, cursor: str = None):
        
        response = self.session.get_positions(
            category=market,
            symbol=ticker,
            settleCoin=settleCoin,
            limit=limit,
            cursor=cursor
        )

        return utils.parse_positions(response)
    

    

In [288]:
wrapper=BybitWrapper(demo=False)
test_wrapper=BybitWrapper(demo=True)

Wrapper Activated. Demo Mode == False
Wrapper Activated. Demo Mode == True


### Account Data

In [ ]:
#wrapper.get_coin_balance() # not available in demo mode 

In [281]:
test_wrapper.transaction_log()

/var/folders/p_/s5m9ntfs6rs0ntx6z7795qgrsb4pj6/T/ipykernel_40032/4020974685.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_transactions = pd.concat([all_transactions, page_transactions], ignore_index=True)


,order_link_id,symbol,category,side,transaction_time,type,qty,size,currency,trade_price,funding,fee,cash_flow,change,cash_balance,fee_rate,bonus_change,trade_id,order_id,id
0,,USDCUSDT,spot,Sell,2024-08-12 20:03:43,TRADE,-50000.0000,0.000,USDC,0.9998,0.000000,0.000000e+00,-50000.000000,-50000.000000,0.000000,0.000000,0.0,1721887545796251013,1751180839552814336,249594355_22001_552058639_1
1,,USDCUSDT,spot,Sell,2024-08-12 20:03:43,TRADE,49990.0000,0.000,USDT,0.9998,0.000000,0.000000e+00,49990.000000,49990.000000,103800.937174,0.000000,0.0,1721887545796251013,1751180839552814336,249594355_22001_552058639_0
2,Buy_0-092_BTCUSDT_20240812_161421,BTCUSDT,linear,Buy,2024-08-12 19:14:23,TRADE,0.0920,0.000,USDT,58965.9000,0.000000,2.983675e+00,-12.111800,-15.095475,53810.937174,0.000550,0.0,72c69793-4011-4aa0-90b7-c6e295f7cd67,84107c2f-7b65-4e4f-970e-44f4dbf62428,249594355_BTCUSDT_587504539_0
3,Sell_0-046_BTCUSDT_20240812_161240,BTCUSDT,linear,Sell,2024-08-12 19:12:41,TRADE,0.0460,-0.092,USDT,59002.4000,0.000000,1.492761e+00,0.000000,-1.492761,53826.032649,0.000550,0.0,448cc415-0bad-439f-b70e-e2a8db9de056,e8a50898-1766-4d4f-8ac7-dc061cf0becd,249594355_BTCUSDT_587502413_0
4,Buy_1_ETHUSDT_20240812_160051,ETHUSDT,linear,Buy,2024-08-12 19:00:53,TRADE,1.0000,0.000,USDT,2634.5100,0.000000,1.448981e+00,0.940000,-0.508980,53827.525409,0.000550,0.0,f95262bc-1969-4df1-8061-2c2255cd001f,fa98b219-f31f-4d15-82a0-55751e13ed2f,249594355_ETHUSDT_587486980_0
5,Sell_1_ETHUSDT_20240812_160026,ETHUSDT,linear,Sell,2024-08-12 19:00:28,TRADE,1.0000,-1.000,USDT,2635.4500,0.000000,1.449498e+00,0.000000,-1.449498,53828.034390,0.000550,0.0,93773ec0-16f6-4a8d-abc7-780d468926e7,4904b5fe-460d-4a56-9cd5-9b2396a606a7,249594355_ETHUSDT_587486420_0
6,Sell_8_ETHUSDT_20240812_143749,ETHUSDT,linear,Sell,2024-08-12 17:37:50,TRADE,8.5300,0.000,USDT,2674.6700,0.000000,1.254821e+01,918.339800,905.791586,53829.483887,0.000550,0.0,a697cecb-306b-4871-b622-ee3387223ecc,0b82b5cf-0869-4c29-a450-334494fa23ff,249594355_ETHUSDT_587377946_0
7,,BTCUSDT,spot,Buy,2024-08-12 16:38:10,TRADE,0.0167,0.000,BTC,59724.5800,0.000000,1.675000e-05,0.016744,0.016727,0.853635,0.001000,0.0,1721887545775223260,1751077383638879488,249594355_22009_552057857_1
8,,BTCUSDT,spot,Buy,2024-08-12 16:38:10,TRADE,-1000.0000,0.000,USDT,59724.5800,0.000000,0.000000e+00,-1000.000000,-1000.000000,52923.692302,0.000000,0.0,1721887545775223260,1751077383638879488,249594355_22009_552057857_0
9,,ETHUSDT,linear,Buy,2024-08-12 16:00:00,SETTLEMENT,8.5300,8.530,USDT,2687.9600,-1.300952,0.000000e+00,0.000000,-1.300952,53923.692302,0.000057,0.0,f84906ff-2a7c-4fa3-83e1-c2a739046259,31bdd727-010e-405f-a3f4-8bc9c255c8e1,249594355_ETHUSDT_587249640_0


In [283]:
test_wrapper.wallet_balance()

,account_type,total_equity,total_wallet_balance,total_margin_balance,total_available_balance,coin,equity,usd_value,wallet_balance,free,locked,spot_hedging_qty,borrow_amount,available_to_withdraw,accrued_interest,unrealised_pnl,cum_realised_pnl,margin_collateral,collateral_switch
0,UNIFIED,161340.976915,103817.649125,103817.649125,103817.649125,BTC,0.853635,50174.955484,0.853635,0.0,0.0,0.0,0.0,0.853635,0.0,0.0,-0.000354,True,False
1,UNIFIED,161340.976915,103817.649125,103817.649125,103817.649125,ETH,2.764510,7348.372306,2.764510,0.0,0.0,0.0,0.0,2.764510,0.0,0.0,-0.001866,True,False
2,UNIFIED,161340.976915,103817.649125,103817.649125,103817.649125,USDT,103800.937174,103817.649125,103800.937174,0.0,0.0,0.0,0.0,103800.937174,0.0,0.0,-131.142342,True,True


In [ ]:
test_wrapper.wallet_balance(account_type='UNIFIED', coin='BTC')

### Market Data

In [ ]:
test_wrapper.get_candles(ticker='BTCUSDT', market='spot')

### Managing SPOT Orders

In [ ]:
# Build the payload
payload = test_wrapper.build_spot_market_order_payload(
    ticker='USDCUSDT', 
    side='Sell', # when side is 'BUY', qty is quoteCoin, when side is 'Sell', qty is baseCoin
    qty=test_wrapper.wallet_balance(coin='USDC')['equity'].item(),
)
payload


In [ ]:
# Place the order using the payload
test_wrapper.place_spot_market_order(payload)


In [ ]:
test_wrapper.spot_order_history()

### Managing PERP Orders

In [ ]:
test_wrapper.perp_order_history()

In [289]:
test_wrapper.leverage(ticker='BTCUSDT', buy_leverage='1', sell_leverage='1')

{'retCode': 0,
 'retMsg': 'OK',
 'result': {},
 'retExtInfo': {},
 'time': 1723677039400}

In [293]:
payload=test_wrapper.build_perp_market_order_payload(ticker='BTCUSDT',
                                             side= 'Buy',
                                             qty=1,
                                             execution_type='IOC',
                                             )
payload

{'category': 'linear',
 'symbol': 'BTCUSDT',
 'side': 'Buy',
 'orderType': 'Market',
 'qty': 1,
 'timeInForce': 'IOC',
 'orderLinkId': 'Buy_1_BTCUSDT_20240814_201326',
 'isLeverage': 0,
 'orderFilter': 'Order'}

In [294]:
test_wrapper.place_perp_market_order(payload) # understand output 

,created_time,order_link_id,side,symbol,avg_price,quantity,order_status,cum_exec_qty,cum_exec_value,cum_exec_fee,...,leaves_value,time_in_force,order_type,trigger_price,take_profit,stop_loss,reduce_only,close_on_trigger,order_id,updated_time
0,2024-08-14 23:13:28,Buy_1_BTCUSDT_20240814_201326,Buy,BTCUSDT,58917.3,1.0,Filled,1.0,58917.3,32.404515,...,0.0,IOC,Market,0.0,0.0,0.0,False,False,71c3bc31-6f25-48fa-9cc6-c31d4f0acc29,2024-08-14 23:13:28


In [295]:
test_wrapper.positions()

,created_time,updated_time,symbol,side,size,avg_price,position_value,unrealised_pnl,leverage,liq_price,mark_price,position_status,trade_mode,position_balance,take_profit,stop_loss,position_idx
0,2024-08-11 18:47:02,2024-08-14 23:13:28,BTCUSDT,Buy,1.0,58917.3,58917.3,14.5,1.0,,58931.8,Normal,0,0.0,None,None,0


In [296]:
test_wrapper.transaction_log().iloc[:,1:15]

/var/folders/p_/s5m9ntfs6rs0ntx6z7795qgrsb4pj6/T/ipykernel_40032/2019075828.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_transactions = pd.concat([all_transactions, page_transactions], ignore_index=True)


,symbol,category,side,transaction_time,type,qty,size,currency,trade_price,funding,fee,cash_flow,change,cash_balance
0,BTCUSDT,linear,Buy,2024-08-14 23:13:28,TRADE,1.0000,1.000,USDT,58917.3000,0.000000,3.240452e+01,0.000000,-32.404515,103768.532659
1,USDCUSDT,spot,Sell,2024-08-12 20:03:43,TRADE,-50000.0000,0.000,USDC,0.9998,0.000000,0.000000e+00,-50000.000000,-50000.000000,0.000000
2,USDCUSDT,spot,Sell,2024-08-12 20:03:43,TRADE,49990.0000,0.000,USDT,0.9998,0.000000,0.000000e+00,49990.000000,49990.000000,103800.937174
3,BTCUSDT,linear,Buy,2024-08-12 19:14:23,TRADE,0.0920,0.000,USDT,58965.9000,0.000000,2.983675e+00,-12.111800,-15.095475,53810.937174
4,BTCUSDT,linear,Sell,2024-08-12 19:12:41,TRADE,0.0460,-0.092,USDT,59002.4000,0.000000,1.492761e+00,0.000000,-1.492761,53826.032649
5,ETHUSDT,linear,Buy,2024-08-12 19:00:53,TRADE,1.0000,0.000,USDT,2634.5100,0.000000,1.448981e+00,0.940000,-0.508980,53827.525409
6,ETHUSDT,linear,Sell,2024-08-12 19:00:28,TRADE,1.0000,-1.000,USDT,2635.4500,0.000000,1.449498e+00,0.000000,-1.449498,53828.034390
7,ETHUSDT,linear,Sell,2024-08-12 17:37:50,TRADE,8.5300,0.000,USDT,2674.6700,0.000000,1.254821e+01,918.339800,905.791586,53829.483887
8,BTCUSDT,spot,Buy,2024-08-12 16:38:10,TRADE,0.0167,0.000,BTC,59724.5800,0.000000,1.675000e-05,0.016744,0.016727,0.853635
9,BTCUSDT,spot,Buy,2024-08-12 16:38:10,TRADE,-1000.0000,0.000,USDT,59724.5800,0.000000,0.000000e+00,-1000.000000,-1000.000000,52923.692302
